In [ ]:
import os
from importlib import reload
from typing import List

import numpy as np
import tensorflow as tf
from serial import Serial

import spectrogram as spectrogram
import spectrum_painting as sp
import spectrum_painting_data as sp_data
import spectrum_painting_model as sp_model
import spectrum_painting_plotting as sp_plot
import spectrum_painting_predict as sp_predict
import spectrum_painting_training as sp_training
from save_iq_data_for_arduino import save_iq_data

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Convert MATLAB data to numpy files to make loading quicker

See `convert_matlab_to_numpy.py` to do call this manually without running it in the notebook.

# Load spectrograms

In [ ]:
reload(sp_training)

classes = ["Z", "B", "W", "BW", "ZB", "ZW", "ZBW"]
snr = 15
spectrum_painting_options = sp_training.SpectrumPaintingTrainingOptions(
    downsample_resolution=64,
    k=3,
    l=16,
    d=4
)

In [ ]:
# Reload spectrum painting module in case the code changed
# and you want what is executed to be what you wrote.
reload(spectrogram)
reload(sp)
reload(sp_data)

spectrograms = sp_data.load_spectrograms(data_dir="data/numpy",
                                         classes=classes,
                                         snr_list=[snr],
                                         windows_per_spectrogram=256,
                                         window_length=256,
                                         nfft=64)

In [ ]:
reload(sp)
reload(spectrogram)
reload(sp_plot)
reload(sp_predict)
reload(sp_training)

train_test_sets = sp_training.create_spectrum_painting_train_test_sets(
    spectrograms=spectrograms,
    label_names=classes,
    options=spectrum_painting_options,
    test_size=0.3
)

print(f"Number of training images: {len(train_test_sets.y_train)}")
print(f"Number of testing images: {len(train_test_sets.y_test)}")

In [ ]:
image_index = 30
sp_plot.plot_spectrogram(train_test_sets.x_test_augmented[image_index], "Augmented")
sp_plot.plot_spectrogram(train_test_sets.x_test_painted[image_index], "Painted")

sp_plot.plot_train_images(train_test_sets.x_test_painted,
                          train_test_sets.y_test,
                          train_test_sets.label_names,
                          train_test_sets.test_snr)

# Create TensorFlow model

In [ ]:
reload(sp_model)
reload(sp_predict)
reload(sp_plot)

image_shape = train_test_sets.x_train_augmented[0].shape

tf_model = sp_model.create_tensorflow_model(image_shape=image_shape, label_count=len(train_test_sets.label_names))
tf_model.summary()
tf.keras.utils.plot_model(tf_model, to_file="output/model.png", dpi=60)

history = sp_model.fit_model(tf_model, train_test_sets, epochs=200, early_stopping_patience=20)

sp_plot.plot_model_accuracy_epochs(history)
sp_plot.plot_model_loss(history)

final_loss, full_model_accuracy = tf_model.evaluate([train_test_sets.x_test_augmented,
                                                     train_test_sets.x_test_painted],
                                                    train_test_sets.y_test,
                                                    verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, full_model_accuracy))

output_file = f"output/spectrum-painting-model.keras"

tf_model.save(output_file, save_format="keras")

keras_model_size = os.stat(output_file).st_size
print(f"Done. Keras model size = {keras_model_size // 1000} KB")

# Predict with full Tensorflow
test_img_augmented = train_test_sets.x_test_augmented[17]
test_img_painted = train_test_sets.x_test_painted[17]

sp_plot.plot_spectrogram(test_img_augmented, "Augmented")
sp_plot.plot_spectrogram(test_img_painted, "Painted")

prediction = sp_predict.predict_full_model(tf_model, test_img_augmented, test_img_painted)

print(train_test_sets.label_names[prediction])

tf_model_y_predictions = [sp_predict.predict_full_model(tf_model, x_a, x_p) for (x_a, x_p) in
                          zip(train_test_sets.x_test_augmented, train_test_sets.x_test_painted)]

sp_plot.plot_confusion_matrix(tf_model_y_predictions, train_test_sets.y_test, train_test_sets.label_names)

# Convert to Lite model

In [ ]:
reload(sp_model)
reload(sp_predict)

tflite_model = sp_model.convert_to_tensorflow_lite(tf_model,
                                                   train_test_sets.x_test_augmented,
                                                   train_test_sets.x_test_painted)
print(f"Done. Model size = {len(tflite_model) // 1000} KB")

lite_output_file = f"output/spectrum-painting-model.tflite"
# Save the model.
with open(lite_output_file, "wb") as f:
    f.write(tflite_model)

# Create a hex dump of the model in a C file for the arduino.
# Use sed to make the variables const since they are huge.
! xxd -i $lite_output_file | sed -e "s/unsigned/const unsigned/"  > ../arduino/spectrum_painting/model.h

image_index = 20

# Test with Tensorflow Lite
tflite_model_y_predictions: List[int] = []

for x_aug, x_painted in list(zip(train_test_sets.x_test_augmented, train_test_sets.x_test_painted)):
    tflite_model_y_predictions.append(sp_predict.predict_lite_model(tflite_model, x_aug, x_painted))

tflite_accuracy = np.sum(train_test_sets.y_test == tflite_model_y_predictions) / len(train_test_sets.y_test)

print(f"Lite model accuracy = {tflite_accuracy}")
sp_plot.plot_confusion_matrix(np.asarray(tflite_model_y_predictions),
                              train_test_sets.y_test,
                              train_test_sets.label_names)

# Save IQ data to the arduino

In [ ]:
iq_data = np.load(f"data/numpy/SNR{snr}_ZBW.npy")

save_iq_data(iq_data, windows=64, window_length=1024, file="../arduino/spectrum_painting/data.h")

# Compile and upload to arduino

In [ ]:
! arduino-cli compile --fqbn arduino:mbed:nano33ble ../arduino/spectrum_painting/spectrum_painting.ino
! arduino-cli upload --fqbn arduino:mbed:nano33ble ../arduino/spectrum_painting/spectrum_painting.ino --port /dev/cu.usbmodem21301

In [ ]:
serial = Serial(port="/dev/cu.usbmodem21301", baudrate=115200, timeout=1000)

arduino_spectrogram = []
for _ in range(spectrum_painting_options.downsample_resolution):
    real_line = serial.readline().decode('utf-8').strip()
    magnitudes = list(map(float, real_line.split(',')))

    arduino_spectrogram.append(magnitudes)

downsample_duration = int(serial.readline().strip())
augment_duration = int(serial.readline().strip())
paint_duration = int(serial.readline().strip())
digitize_duration = int(serial.readline().strip())
inference_duration = int(serial.readline().strip())
total_duration = int(serial.readline().strip())
predicted_label = int(serial.readline().strip())

sp_plot.plot_spectrogram(np.asarray(arduino_spectrogram), "Arduino spectrogram")

print(f"Downsample duration = {downsample_duration} ms")
print(f"Augment duration = {augment_duration} ms")
print(f"Paint duration = {paint_duration} ms")
print(f"Digitize duration = {digitize_duration} ms")
print(f"Inference duration = {inference_duration} ms")
print(f"Total duration = {total_duration} ms")
print(f"Predicted label = {classes[predicted_label]}")
print("\n\n")
print(f"Full keras model size = {keras_model_size // 1000} KB")
print(f"Lite model size = {len(tflite_model) // 1000} KB")
print(f"Full model accuracy = {full_model_accuracy}")
print(f"Lite model accuracy = {tflite_accuracy}")

serial.close()